In [1]:
from config import API_Key
import pandas as pd
import requests
import json
from citipy import citipy

In [5]:
url = f"http://api.airvisual.com/v2/city?city=Los Angeles&state=California&country=USA&key={API_Key}"
response=requests.get(url)
data=response.json()
print(json.dumps(data,indent=4,sort_keys=4))

{
    "data": {
        "city": "Los Angeles",
        "country": "USA",
        "current": {
            "pollution": {
                "aqicn": 33,
                "aqius": 73,
                "maincn": "p2",
                "mainus": "p2",
                "ts": "2022-06-16T08:00:00.000Z"
            },
            "weather": {
                "hu": 85,
                "ic": "50n",
                "pr": 1009,
                "tp": 18,
                "ts": "2022-06-16T08:00:00.000Z",
                "wd": 130,
                "ws": 3.09
            }
        },
        "location": {
            "coordinates": [
                -118.2417,
                34.0669
            ],
            "type": "Point"
        },
        "state": "California"
    },
    "status": "success"
}


In [11]:
filepath="Climate_Data/ClimateClassifications.csv"
climate_df=pd.read_csv(filepath)
climate_df

,Lat,Lon,Cls
0,-89.75,-179.75,EF
1,-89.75,-179.25,EF
2,-89.75,-178.75,EF
3,-89.75,-178.25,EF
4,-89.75,-177.75,EF
...,...,...,...
92411,83.75,-30.75,ET
92412,83.75,-30.25,ET
92413,83.75,-29.75,ET
92414,83.75,-29.25,ET


In [47]:
nearest_city=[]
nearest_city_df=climate_df
for i in range(0,len(nearest_city_df)):
    lat=nearest_city_df.loc[i,"Lat"]
    lon=nearest_city_df.loc[i,"Lon"]
    city=citipy.nearest_city(lat,lon)
    nearest_city.append(city.city_name)
    
    


In [50]:
climate_cities=nearest_city_df
climate_cities["Nearest City"]=nearest_city
climate_cities
climate_cities.to_csv("Climate_Data/Nearest_Cities.csv")

In [53]:
cities_shortlist_df=climate_cities.drop_duplicates(subset=["Nearest City"],keep='first')
cities_shortlist_df.to_csv("Climate_Data/Nearest_Cities_Shortlist.csv")
cities_shortlist_df

,Lat,Lon,Cls,Nearest City
0,-89.75,-179.75,EF,vaini
24,-89.75,-167.75,EF,mataura
78,-89.75,-140.75,EF,rikitea
106,-89.75,-126.75,EF,punta arenas
176,-89.75,-91.75,EF,ushuaia
...,...,...,...,...
86811,70.75,79.25,ET,dikson
87714,72.25,-87.75,ET,qaanaaq
89074,74.75,19.25,ET,longyearbyen
89483,75.75,-67.25,ET,narsaq
